# Mapreader Workshops 2024

----

First check you have the correct version of MapReader: v1.3.2

This can be downloaded from pypi using `pip install mapreader==1.3.2` or by checking out the repo at [this commit](https://github.com/Living-with-machines/MapReader/releases/tag/v1.3.2)

In [2]:
import mapreader
assert mapreader.__version__ == '1.3.2'

-------------

# Classify

We can also use our ``ClassifierContainer`` to infer labels on new datasets with a pre-trained/fine-tuned model.

We are going to look at how to use our railspace model (https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace) to predict labels on patches.To do this, we need to load the model from huggingface and then pass it in as the ``model`` argument in our ``ClassifierContainer``.

We need to use the ``timm`` library to set up our model. You can see this is shown in the top right corner of the [model page](https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace) where it says ``</> Use in timm``.

Other models will have different instructions, e.g. [resnet-50](https://huggingface.co/microsoft/resnet-50) should be loaded with the ``transformers`` library.

In [3]:
try:
    import timm
except ImportError: # timm isn't installed by default, so you might need to install it
    !pip install timm
    import timm

In [4]:
my_model = timm.create_model("hf_hub:Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_building", pretrained=True)

config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/186M [00:00<?, ?B/s]

After setting up the model, we can then set up our ``ClassifierContainer``. This is done in the same way as before, except this time we pass ``my_model`` as the model argument.

In [ ]:
from mapreader import ClassifierContainer

The below will make sure that the model training/inference runs as as fast as possible on your machine by using CUDA (GPU) or MPS if they are available.

This ``device`` variable can then be fed into the ``ClassifierContainer``.

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'  

In [ ]:
my_classifier = ClassifierContainer(
    model=my_model,
    labels_map={0:"no", 1:"railspace"}, # manually set this, you can find it in the model card - https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace
    dataloaders=None,
)

## Infer

The model can now be used to infer, or predict, the labels of "unseen" patches.

To show how inference works, we will predict the labels on patches from just one parent image. 

We will do this by creating a ``subset_patch_df`` from our previously saved ``patch_df.csv``.
Our new ``subset_patch_df`` will only contain the information of patches from ``map_75650661.png``.

In [ ]:
import pandas as pd

patch_df = pd.read_csv("./patch_df.csv", index_col=0)  # load our patch_df.csv file

subset_patch_df = patch_df[
    patch_df["parent_id"] == "map_75650661.png"
]  # filter for our chosen parent image
subset_patch_df.head()

> __**NOTE**__: MapReader can be used to predict the labels on entire datasets and so creating a ``subset_patch_df`` is not needed in most use cases.

### Create a dataset (``infer``) from our ``subset_patch_df``

In [ ]:
from mapreader import PatchDataset

In [ ]:
infer = PatchDataset(subset_patch_df, transform="val", patch_paths_col="image_path")

### Load dataset into ``my_classifier``

In the same way as we did in the last notebook, we need to load our dataset into our classifier container.

In [ ]:
my_classifier.load_dataset(infer, "infer_railspace")

### Run model inference

__**YOUR TURN**__: Run inference on your ``"infer_railspace"`` dataset

See [here](https://mapreader.readthedocs.io/en/latest/User-guide/Classify/Infer.html#infer) in docs.

In [ ]:
# my_classifier.inference()

Remember to save your results!

In [ ]:
my_classifier.save_predictions("infer_railspace")

### Save results to metadata

To add the predictions back into a ``MapImages`` object, we simply need to load our predictions csv file as metadata.

Since we have started a new notebook, we can create a new ``MapImages`` object by loading our patches.

In [ ]:
from mapreader import load_patches

In [ ]:
my_maps = load_patches(
    "./patches_100_meters/*75650661*png", parent_paths="./maps/*75650661.png"
)

In [ ]:
my_maps.add_metadata("./infer_railspace_predictions_patch_df.csv", tree_level="patch")

In [ ]:
my_maps.add_shape()

We can use the ``.show_parent()`` method to see how our predictions look on our parent map sheet (``map_74488700.png``).

In [ ]:
my_maps.show_parent(
    "map_75650661.png",
    column_to_plot="pred",
    vmin=0,
    vmax=1,
    alpha=0.5,
    patch_border=False,
)

And the ``.convert_images()`` method to save our results.

In [ ]:
parent_df, patch_df = my_maps.convert_images(save=True, save_format="excel") # here we are saving to xlsx so we don't change our "*.csv" files from before!

We can also save our outputs as a ``geojson`` file using the ``.save_patches_to_geojson()`` method. We'll call the file ``"railspace_patches.geojson"``.

> _**NOTE**_: This will require you to convert your patch coordinates into a polygon format. If these aren't already available, they can be added using the ``.add_patch_polygons()`` method.

In [ ]:
my_maps.add_patch_polygons()
my_maps.save_patches_to_geojson("railspace_patches.geojson")

Beyond MapReader, these outputs can be used to generate interesting visualizations in other tools.

For example, here are two visualizations of the rail space data from [our paper]:

- https://felt.com/map/MapReader-Launch-Event-map-Urban-Areas-and-Rail-space-9AqftKrvPTlWfwOGkdkCGkD
- https://maps.nls.uk/projects/mapreader/index.html#zoom=6.0&lat=56.00000&lon=-4.00000

# Documentation

Please refer to the [MapReader documentation](https://mapreader.readthedocs.io/en/latest/) for more information.